# Dev Notebook: Running Process Functions

**Author:** Xavier Nogueira

**Problem:** As of October 11th 2023 we use `xarray.apply_ufunc` to map equations unto our `xarray.dataset` to iterate timesteps. This works with math only equations, where operators combine arrays, however if/else decision tree logic does not work on the arrays.

**Solution:** Explore a way to support both equation versions in this notebook, while optimizing for model timestep execution performance.

In [1]:
import xarray as xr
import numpy as np
import numba

# Create mock data

## Add variables to the air temp xarray

In [2]:
air_ds = xr.tutorial.open_dataset('air_temperature')
air_ds['air2'] = air_ds.air * 2
air_ds['air3'] = air_ds.air * 3
air_ds = air_ds.isel(time=-1)
air_ds

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
    time     datetime64[ns] 2014-12-31T18:00:00
Data variables:
    air      (lat, lon) float32 245.1 244.3 243.3 242.2 ... 296.5 296.2 295.7
    air2     (lat, lon) float32 490.2 488.6 486.6 484.4 ... 593.0 592.4 591.4
    air3     (lat, lon) float32 735.3 732.9 729.9 726.6 ... 889.5 888.6 887.1
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## Create mock functions

In [3]:
@numba.njit
def mock_1(air, air2, air3):
    return air * 0.01 * air2 * air3

@np.vectorize
def mock_1_vec(air, air2, air3):
    return air * 0.01 * air2 * air3


def mock_2(air, air2, air3):
    combo = air * 0.01 * air2 * air3
    return xr.where(combo > 1243597, 1, 0)

@numba.njit
def mock_2_loop(air, air2, air3):
    result = np.zeros_like(air)  # Create an array of zeros with the same shape as 'air'
    for i in range(air.shape[0]):
        for j in range(air.shape[1]):
            combo = mock_1(air[i, j], air2[i, j], air3[i, j])
            if combo > 1243597:
                result[i, j] = 1
    return result

# Test them out

**Findings:**
* Using the simple equation of "mock 1", pre-vectorizing vs JIT-compiling produce relatively simulat speeds with JIT seeming to be a tad faster. This gap will likely increase with iterations, so **numba is still the better choice**.
* Regarding IF/ELSE logic, one can either use `xr.where` or loop thru all indices and output a fresh numpy array.
* Using `xr.where` DO NOT use `np.vectorize` (it 10x slows it wierdly), and JIT does not work. Regardless it is reasonably fast.
* Using the loop we can JIT compile it, and it **seems slightly faster**. That said, the logic is more complex.

In [4]:
input_list: list[xr.DataArray] = [air_ds.air, air_ds.air2, air_ds.air3]

In [5]:
%%timeit
air_ds['mock_1_numba'] = xr.apply_ufunc(
    mock_1,
    *input_list
)
air_ds['mock_1_numba']

3.58 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
air_ds['mock_1_np'] = xr.apply_ufunc(
    mock_1_vec,
    *input_list
)
air_ds['mock_1_np']

3.55 ms ± 138 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
air_ds['mock_1_np'].quantile(q=0.5)

<xarray.DataArray 'mock_1_np' ()>
array(1243597.4133213)
Coordinates:
    quantile  float64 0.5

In [8]:
%%timeit
air_ds['mock_2'] = xr.apply_ufunc(
    mock_2,
    *input_list
)
air_ds['mock_2']

3.84 ms ± 256 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
air_ds['mock_2_loop'] = xr.apply_ufunc(
    mock_2_loop,
    *input_list
)
air_ds['mock_2_loop']

4.03 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
